In [222]:
import pandas as pd
import numpy as np
import os
import re
import math

In [223]:
country_list = pd.read_csv('data/countries.csv')
# change name of the Code column to match datasets, change Label to country
country_list = country_list.rename(columns={'Code': 'iso3', 'Label': 'country'})
country_list.head()

,country,iso3
0,Afghanistan,AFG
1,Angola,AGO
2,Albania,ALB
3,Andorra,AND
4,United Arab Emirates,ARE


In [224]:
# number of unique countries
country_list['iso3'].nunique()

188

In [225]:
# countries with nan values only: Andorra (AND), North Korea (PRK), Somalia (SOM)
nan_isos = ['AND', 'PRK', 'SOM']
for iso in nan_isos:
    country_list = country_list[country_list.iso3 != iso]

country_list['iso3'].nunique()

185

In [226]:
country_list.to_csv('final_datasets/all_countries.csv', index=False)

In [227]:
def load_and_prepare_dataset(filename):
    # load the data
    if os.path.isdir('data/nutrition'):
        print('***********************')
        print('Loading dataset ' + filename + '...\n')
        path = 'data/nutrition/' + filename
        dataset = pd.read_csv(path)
    else:
        print('Directory not found')
        
        
    # fix missing varnum column
#     missing_varnums = ['v10_cnty.csv', 'v33_cnty.csv', 'v35_cnty.csv']
    missing_varnums = ['10', '33', '35', '50']

    for varnum in missing_varnums:
        if varnum in filename:
            print('adding varnum column to ', varnum)
            dataset['varnum'] = varnum


    print('Sample before manipulation: ')
    print(dataset.head())
  
    print('\n\nShape before manipulation: ')
    print(dataset.shape)
    
    print('\n\nColumns before manipulation: ')
    print(dataset.count())
    
    # check unique iso3 codes in both dataset and country_list
    unique_iso_in_country_lst = country_list['iso3'].unique()
    print('\nUnique countries in country list:', len(unique_iso_in_country_lst))
    unique_iso_in_data = dataset['iso3'].unique()
    print('Unique countries in dataset:', len(unique_iso_in_data))
    
    # find the countries that are in iso list but not in the dataset
    countries_not_in_dataset = list(set(unique_iso_in_country_lst) - set(unique_iso_in_data))
    print('\nCountries in country list but not in the dataset', filename, countries_not_in_dataset)
    
    # find the countries that are in the data but not in iso list
    countries_not_in_country_list = list(set(unique_iso_in_data) - set(unique_iso_in_country_lst))
    print('Countries NOT in country list ', countries_not_in_country_list)
    
    # find the total estimates for each country and year
    filtered_dataset = dataset[ dataset['age'].eq(999) & dataset['female'].eq(999) & dataset['urban'].eq(999) & dataset['edu'].eq(999) ]

    # typecast age and year as integer
    filtered_dataset = filtered_dataset.astype({"age": int, "year": int})
    
    # get all columns from the dataset:
    columns_list = list(filtered_dataset.columns)
    print('columns_list', columns_list)
    
    # change columns with values names to contain the variable code (varnum)
    var_type = re.findall(r'\d+', filename)[0]
    var_name = 'v' + var_type 
    
    print('var_name', var_name)
    
    for column in columns_list:
        if var_name in column:
            print(column)
            new_name = column.replace(var_name, '')
            filtered_dataset.rename(columns={column: new_name}, inplace=True)
            print(column)
  
    # rename columns
    filtered_dataset.rename(columns={'superregion2': 'region', 'var': 'nutrient_code', '_type': 'type', '_type_desc': 'type_desc'}, inplace=True)

    # remove unnecessary columns from data
    filtered_dataset = filtered_dataset.drop(columns=['age', 'female', 'urban', 'edu'])

#     if 'serving', 's_lowerci_95', 's_upperci_95' exist, drop them form the dataset
    if 'serving' in filtered_dataset.columns:
        filtered_dataset = filtered_dataset.drop(columns=['serving', 's_lowerci_95', 's_upperci_95'])
    
    print('\n\nShape after manipulation: ')
    print(filtered_dataset.shape)
    
    print('\nDataset ' + filename + ' processed successfully!\n')
    print('***********************\n')
    
    clean_data_directory = 'clean_data/clean_nutrition'
    file_name = 'cleaned_' + filename
    
    if os.path.isdir(clean_data_directory):
        # save file to directory
        filtered_dataset.to_csv(clean_data_directory + '/' + file_name)
        
    else:
        # create directory, save file
        os.mkdir(clean_data_directory)
        filtered_dataset.to_csv(clean_data_directory + '/' + file_name)
        
#     filtered_dataset
    return filtered_dataset

In [228]:
test = list() 

for filename in os.listdir("data/nutrition"):
#     print(filename)
    if filename[0] == 'v':
        test.append(load_and_prepare_dataset(filename))


***********************
Loading dataset v05_cnty.csv...


Unique countries in country list: 185
Unique countries in dataset: 185

Countries in country list but not in the dataset v05_cnty.csv []
Countries NOT in country list  []
columns_list ['superregion2', 'iso3', 'age', 'female', 'urban', 'edu', 'year', 'varnum', 'v05_type', 'v05_type_desc', 'median', 'upperci_95', 'lowerci_95', 'serving', 's_lowerci_95', 's_upperci_95']
var_name v05
v05_type
v05_type
v05_type_desc
v05_type_desc

Dataset v05_cnty.csv processed successfully!

***********************

***********************
Loading dataset v04_cnty.csv...


Unique countries in country list: 185
Unique countries in dataset: 185

Countries in country list but not in the dataset v04_cnty.csv []
Countries NOT in country list  []
columns_list ['superregion2', 'iso3', 'age', 'female', 'urban', 'edu', 'year', 'varnum', 'v04_type', 'v04_type_desc', 'median', 'upperci_95', 'lowerci_95', 'serving', 's_lowerci_95', 's_upperci_95']
var_name v04



Unique countries in country list: 185
Unique countries in dataset: 185

Countries in country list but not in the dataset v03_cnty.csv []
Countries NOT in country list  []
columns_list ['superregion2', 'iso3', 'age', 'female', 'urban', 'edu', 'year', 'varnum', 'v03_type', 'v03_type_desc', 'median', 'upperci_95', 'lowerci_95', 'serving', 's_lowerci_95', 's_upperci_95']
var_name v03
v03_type
v03_type
v03_type_desc
v03_type_desc

Dataset v03_cnty.csv processed successfully!

***********************

***********************
Loading dataset v41_cnty.csv...


Unique countries in country list: 185
Unique countries in dataset: 185

Countries in country list but not in the dataset v41_cnty.csv []
Countries NOT in country list  []
columns_list ['superregion2', 'iso3', 'age', 'female', 'urban', 'edu', 'year', 'varnum', 'v41_type', 'v41_type_desc', 'median', 'upperci_95', 'lowerci_95']
var_name v41
v41_type
v41_type
v41_type_desc
v41_type_desc

Dataset v41_cnty.csv processed successfully!

*******


Unique countries in country list: 185
Unique countries in dataset: 185

Countries in country list but not in the dataset v39_cnty.csv []
Countries NOT in country list  []
columns_list ['superregion2', 'iso3', 'age', 'female', 'urban', 'edu', 'year', 'varnum', 'v39_type', 'v39_type_desc', 'median', 'upperci_95', 'lowerci_95']
var_name v39
v39_type
v39_type
v39_type_desc
v39_type_desc

Dataset v39_cnty.csv processed successfully!

***********************

***********************
Loading dataset v42_cnty.csv...


Unique countries in country list: 185
Unique countries in dataset: 185

Countries in country list but not in the dataset v42_cnty.csv []
Countries NOT in country list  []
columns_list ['superregion2', 'iso3', 'age', 'female', 'urban', 'edu', 'year', 'varnum', 'v42_type', 'v42_type_desc', 'median', 'upperci_95', 'lowerci_95']
var_name v42
v42_type
v42_type
v42_type_desc
v42_type_desc

Dataset v42_cnty.csv processed successfully!

***********************

***********************
L

In [229]:
df_row_merged = pd.concat(test, ignore_index=True)
df_row_merged


//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """Entry point for launching an IPython kernel.


,iso3,lowerci_95,median,region,type,type_desc,upperci_95,varnum,year
0,AFG,40.385562,83.317104,SAARC,1,1: Foods & Beverages,168.622554,5,1990
1,AFG,34.698625,71.775725,SAARC,1,1: Foods & Beverages,148.634712,5,1995
2,AFG,20.415994,42.381433,SAARC,1,1: Foods & Beverages,91.962824,5,2000
3,AFG,29.447248,60.322418,SAARC,1,1: Foods & Beverages,125.342277,5,2005
4,AFG,52.585198,104.545995,SAARC,1,1: Foods & Beverages,194.380218,5,2010
5,AFG,50.956284,106.085897,SAARC,1,1: Foods & Beverages,205.395137,5,2015
6,AFG,50.546712,103.830266,SAARC,1,1: Foods & Beverages,197.538241,5,2018
7,AGO,20.247857,28.964633,SSA,1,1: Foods & Beverages,43.687681,5,1990
8,AGO,26.775923,38.153402,SSA,1,1: Foods & Beverages,57.282848,5,1995
9,AGO,19.896019,28.666511,SSA,1,1: Foods & Beverages,42.463388,5,2000


In [230]:
done = pd.merge(df_row_merged, country_list, on="iso3", how='left')

In [231]:
done['type_desc'] = done['type_desc'].str.replace(r'\d+(?=:)', '')
done['type_desc'] = done['type_desc'].str.replace(r'\:', '')

In [232]:
done

,iso3,lowerci_95,median,region,type,type_desc,upperci_95,varnum,year,country
0,AFG,40.385562,83.317104,SAARC,1,Foods & Beverages,168.622554,5,1990,Afghanistan
1,AFG,34.698625,71.775725,SAARC,1,Foods & Beverages,148.634712,5,1995,Afghanistan
2,AFG,20.415994,42.381433,SAARC,1,Foods & Beverages,91.962824,5,2000,Afghanistan
3,AFG,29.447248,60.322418,SAARC,1,Foods & Beverages,125.342277,5,2005,Afghanistan
4,AFG,52.585198,104.545995,SAARC,1,Foods & Beverages,194.380218,5,2010,Afghanistan
5,AFG,50.956284,106.085897,SAARC,1,Foods & Beverages,205.395137,5,2015,Afghanistan
6,AFG,50.546712,103.830266,SAARC,1,Foods & Beverages,197.538241,5,2018,Afghanistan
7,AGO,20.247857,28.964633,SSA,1,Foods & Beverages,43.687681,5,1990,Angola
8,AGO,26.775923,38.153402,SSA,1,Foods & Beverages,57.282848,5,1995,Angola
9,AGO,19.896019,28.666511,SSA,1,Foods & Beverages,42.463388,5,2000,Angola


In [235]:
nulls = done[ done['varnum'].isnull() ]
nulls




,iso3,lowerci_95,median,region,type,type_desc,upperci_95,varnum,year,country


In [ ]:
nulls = list(done['varnum'].unique())
np.sort(nulls)

In [234]:
done.to_csv('data/ingestion/nutrients.csv', encoding='utf-8')
